In [3]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from collections import defaultdict
import time
import psycopg2
from user_agent import generate_user_agent
import numpy as np
import string

# Using Pandas

In [2]:
#grab response
response = requests.get("https://forums.psychcentral.com/depression")
response

<Response [200]>

In [3]:
response = requests.get("https://forums.psychcentral.com/anxiety-panic-phobias/")

In [4]:
soup = BeautifulSoup(response.text, "html.parser")

In [10]:
form = soup.find_all("form")

In [50]:
form = soup.find("form", {"method" : "post", "id": "inlinemodform"})

In [54]:
tr = form.find_all("tr")
r = [row for row in tr if not row.find("img", {"title": "Sticky"})]

In [56]:
len(tr)

28

In [ ]:
def parse_forum(html):
    # get request for forum 
    response = get_request(html)
    
    #create a soup object of request
    soup = soupify(response)
    
    #get links for all threads
    threads = parse_thread_page(soup)
    
    #add all threads and sub-threads to the database
    add_to_db(threads)
    
    #see if a navigation bar exists
    nav_page = soup.find('div', {'class': 'pagenav'})
        # if a next button exists, get the link for the next thread page
        if nav_page != None:
            next_page = soup.find("a", {"rel": "next"})
            
            # while a next button exits, loop through all thread pages and add each thread (and its sub_threads)
            # to the database
            while next_page:
                href = next_page['href']
                response = get_request(href)
                soup = soupify(response)
                threads = parse_thread_page(soup)
                add_to_db(threads)
                next_page = soup.find("a", {"rel": "next"})
    
        
        
    
    
    

In [5]:
#grab hyperlinks and titles of thread posts
def parse_thread_page(soup):
    threads = []
    form = soup.find("form", {"method" : "post", "id": "inlinemodform"})
    tds = form.find_all("td", {"id": re.compile("td_threadtitle_\d+")})
    for td in tds:
        if td.find("img", {"alt": "Sticky Thread"}):
            continue
        else:
            title = td.find("a").text
            hyperlink = td.find("a")['href']
            threads.append({"thread": title, "link": hyperlink})
    return threads
                

# threads_df = pd.DataFrame(threads)
# threads_df.head(20)

In [68]:
#grab hyperlinks and titles of thread posts
def parse_thread_page(soup):
    threads = []
    form = soup.find("form", {"method" : "post", "id": "inlinemodform"})
    tds = form.find_all("td", {"id": re.compile("td_threadtitle_\d+")})
    for td in tds[0:2]:
        return(td)
                

# threads_df = pd.DataFrame(threads)
# threads_df.head(20)

In [51]:
# for each thread, go to each link and grab information

def parse_threads(threads):
    """
    Takes in a list of dictionaries
    Returns 2 pandas tables
    _________________________
    Input:
        - links: list of html links for the entire thread
        - thread: title of the thread
        
    Output:
        - user pandas df 
        - post pandas df
    """
    user_df = pd.DataFrame({"member_since": [], "num_posts": [], "users": []})
    post_df = pd.DataFrame({"posts": [], "post_type": [], "users": []})
    # for each thread, connect to the page, get user and post information
    for thd in threads:
        link = thd['link']
        response = get_request(link)
        soup = soupify(response)
        tables = get_tables(soup)
        user_tbl = make_user_pd(tables)
        post_tbl = get_posts(tables)
        post_tbl['url'] = link
        user_df = pd.concat([user_df, user_tbl])
        post_df = pd.concat([post_df, post_tbl]) 
        # check if a thread has more than one page
        nav_page = soup.find('div', {'class': 'pagenav'})
        # if a thread has more than one page, connect to the next page and update user and post information 
        # until no more pages are left 
        if nav_page != None:
            next_page = soup.find("a", {"rel": "next"})
            while next_page != None:
                href = next_page['href']
                response = get_request(href)
                soup = soupify(response)
                tables = get_tables(soup)
                user_tbl = make_user_pd(tables)
                post_tbl = get_posts(tables)
                post_tbl['url'] = link
                user_df = pd.concat([user_df, user_tbl])
                post_df = pd.concat([post_df, post_tbl])
                next_page = soup.find("a", {"rel": "next"})
        # once the entire thread has been parsed and data uploaded, sleep for a random time (2-5 s) and 
        # repeat for the next thread 
        time.sleep(np.random.randint(2,6))
    user_df.reset_index(drop = True)
    post_df.reset_index(drop = True)
    return user_df, post_df
           
            
        

In [31]:
user_df, post_df = parse_threads(threads[1:3])

https://forums.psychcentral.com/depression/498539-looking-accountability-partners-motivate-each-other-get-things-done-while-depressed.html
https://forums.psychcentral.com/depression/498539-looking-accountability-partners-motivate-each-other-get-things-done-while-depressed-2.html
https://forums.psychcentral.com/depression/498539-looking-accountability-partners-motivate-each-other-get-things-done-while-depressed-3.html
https://forums.psychcentral.com/depression/498539-looking-accountability-partners-motivate-each-other-get-things-done-while-depressed-4.html
https://forums.psychcentral.com/depression/498539-looking-accountability-partners-motivate-each-other-get-things-done-while-depressed-5.html
https://forums.psychcentral.com/depression/498539-looking-accountability-partners-motivate-each-other-get-things-done-while-depressed-6.html
https://forums.psychcentral.com/depression/498539-looking-accountability-partners-motivate-each-other-get-things-done-while-depressed-7.html
https://forums.

https://forums.psychcentral.com/depression/498539-looking-accountability-partners-motivate-each-other-get-things-done-while-depressed-59.html
https://forums.psychcentral.com/depression/498539-looking-accountability-partners-motivate-each-other-get-things-done-while-depressed-60.html
https://forums.psychcentral.com/depression/498539-looking-accountability-partners-motivate-each-other-get-things-done-while-depressed-61.html
https://forums.psychcentral.com/depression/498539-looking-accountability-partners-motivate-each-other-get-things-done-while-depressed-62.html
https://forums.psychcentral.com/depression/498539-looking-accountability-partners-motivate-each-other-get-things-done-while-depressed-63.html
https://forums.psychcentral.com/depression/498539-looking-accountability-partners-motivate-each-other-get-things-done-while-depressed-64.html
https://forums.psychcentral.com/depression/498539-looking-accountability-partners-motivate-each-other-get-things-done-while-depressed-65.html
https:

In [49]:
post_df['posts'].iloc[0]


'As it states in the title, I am looking for accountability partners so that we can all motivate each other to get **** done, set and achieve our goals. We would have daily check ins. \nI, like many others, suffer from a constant funeral in my brain and I need help navigating life and a community to turn corners with. I know that I am not the only one who lacks motivation and lacking the tools and resources in getting out of bed.  \nI am reaching out to PC because I don\x92t have anyone in my circle who can be an accountability partner. Well, I don\x92t have a circle.   \nI am hoping that this will motivate others to get stuff done while lacking motivation to do so. \nI will start...\nI was planning on working out... but I did not... should be attending to my resume and working but I am not...'

In [24]:
post_df = post_df.reset_index(drop = True)
pd.set_option("display.max_columns", None)
post_df

In [45]:
# make a general user table 

def make_user_pd(tables):
    """
    Takes in a list of html elements 
    Returns a dictionary containing
    ____________________
    users: list of usernames
    member_date: list dates became a member for each user
    num_posts: list of total number of posts for each user
    
    """
    users = []
    user_tbl = defaultdict(list)
    for table in tables:
        if  "moderator" in table.find('div', {'class': 'smallfont'}).text.lower():
            continue
        n = 0
        if table.find("a", {"class": "bigusername"}) == None:
            user = table.find("div", {"id": re.compile("postmenu_\d+")}).text.strip()
        else:
            user = table.find("a", {"class": "bigusername"}).text
        if user not in users:
            if table.find_all("div", {"class":"smallfont"})[1].find("img") != None:
                n = 1   
            if table.find_all("div", {"class":"smallfont"})[0].text.lower() == "guest":
                member_date = "nan"
            else:
                member_date = table.findAll('br')[n].findNext('div').text.split(":")[-1].strip()
            if 'location' in table.findAll('br')[n].findAllNext('div')[1].text.lower():
                num_posts = table.findAll('br')[n].findAllNext('div')[2].text
                num_posts = re.sub("[^0-9]", "", num_posts)
            else:
                num_posts = table.findAll('br')[n].findAllNext('div')[1].text
                num_posts = re.sub("[^0-9]", "", num_posts)
            if len(num_posts) == 0:
                num_posts = "nan"
            users.append(user)
            user_tbl['users'].append(user)
            user_tbl['member_since'].append(member_date)
            user_tbl["num_posts"].append(num_posts)
    user_pd = pd.DataFrame(user_tbl)
    return user_pd
        

In [47]:
# make a document table

def get_posts(tables):
    """
    Takes in a list of html elements
    Returns a dictionary
    ____________________
    users: username for each user
    posts: post for each user
    
    """
##### pandas row will be 1 post ######
    post_tbl = defaultdict(list)
    for table in tables:
        if table.find("a", {"class": "bigusername"}) == None:
            user = table.find("div", {"id": re.compile("postmenu_\d+")}).text.strip()
        else:
            user = table.find("a", {"class": "bigusername"}).text
        post = table.find_all("div", {"id": re.compile("post_message_\d+")})[0].text.strip()
        if "http" in post:
            post = re.sub(r'http\S+', '', post)
        post_tbl["users"].append(user) 
        post_tbl["posts"].append(post)
        post_tbl["post_type"].append(["author"] + ["responder"]*(len(post_tbl['users'])-1))
        post_pd = pd.DataFrame(post_tbl)
        return post_pd


###### each pandas row is all posts by user #######
#     ind_post_tbl = defaultdict(list)
#     for table in tables:
#         if table.find("a", {"class": "bigusername"}) == None:
#             user = table.find("div", {"id": re.compile("postmenu_\d+")}).text.strip()
#         else:
#             user = table.find("a", {"class": "bigusername"}).text
#         post = table.find_all("div", {"id": re.compile("post_message_\d+")})[0].text.strip()
#         if "http" in post:
#             post = re.sub(r'http\S+', '', post)
#         ind_post_tbl[user].append(post) 
    
#     post_tbl = defaultdict(list)
#     for user in ind_post_tbl:
#         post_tbl["user"].append(user)
#         post_tbl["posts"].append(ind_post_tbl[user])
#     post_tbl["type_user"] = ["author"] + ["responder"]*(len(ind_post_tbl)-1)
#     post_pd = pd.DataFrame(post_tbl)
#     return post_pd
  

    
    

    

In [13]:
def get_request(url):
    """
    Takes in a url string 
    Outputs a list of html for each user's posts
    """
    
    headers = {"User-Agent": generate_user_agent()}
    response = requests.get(url, headers)
    return response
    

In [7]:
def get_tables(soup):
    tables = soup.find_all("table", {"id": re.compile("post\d+")})
    return tables

In [14]:
response = get_request("https://forums.psychcentral.com/depression/498539-looking-accountability-partners-motivate-each-other-get-things-done-while-depressed.html")

In [17]:
soup = soupify(response)

In [18]:
tbls = get_tables(soup)

In [37]:
pid = tbls[0]["id"]
pid = "".join([letter for letter in pid if letter in string.digits])
pid

'5902375'

In [30]:
tbls[0]

<table align="center" border="0" cellpadding="6" cellspacing="0" class="tborder" id="post5902375" width="100%"> <tr> <td class="thead" style="font-weight:normal; border: 0px solid #CABCAA; border-right: 0px"> <a name="post5902375"><img alt="Old" border="0" class="inlineimg" src="https://pcforums-liviant.netdna-ssl.com/images/styles/pc/statusicon/post_old.gif"/></a>
			11-12-2017, 05:37 PM
			
			 </td> <td align="right" class="thead" style="font-weight:normal; border: 0px solid #CABCAA; border-left: 0px">
			 
			#<a href="https://forums.psychcentral.com/5902375-post1.html" id="postcount5902375" name="1" rel="nofollow" target="new"><strong>1</strong></a> </td> </tr> <tr valign="top"> <td class="alt2" style="border: 0px solid #CABCAA; border-top: 0px; border-bottom: 0px" width="175"> <div id="postmenu_5902375"> <a class="bigusername" href="https://forums.psychcentral.com/member.php?u=435488" rel="nofollow">FallDuskTrain</a> </div> <div class="smallfont">Grand Member</div> <div class="sm

In [16]:
def soupify(response):
    return BeautifulSoup(response.text, "html.parser")

### Using PSQL

**in the terminal**
* `createdb [name]` - creates a new db
* `psql` - connects to the server
* `\l` - shows available dbs
* `\c [dbname]` - connect to another db

In [63]:
# create a connection to the psql server
conn = psycopg2.connect("dbname = mh")

In [64]:
cur = conn.cursor()

In [65]:
# initiate a cursor instance to allow interaction with the database
# Define a query that create a table of user info for psychcentral forums 
create_user_table = """
CREATE TABLE pc_user (
    users varchar(80) PRIMARY KEY,
    num_posts varchar(7),
    member_since varchar(20)
    );
    
"""

create_post_table = """
CREATE TABLE pc_post (
    key   serial primary key,
    users varchar(80),
    posts text,
    post_type varchar(10)
    );"""

cur.execute(create_user_table)
cur.execute(create_post_table)
conn.commit()

In [66]:
def add_to_db(threads):
    """
    Takes in a list of dictionaries
    Returns 2 pandas tables
    _________________________
    Input:
        - links: list of html links for the entire thread
        - thread: title of the thread
        
    Output:
        - user pandas df 
        - post pandas df
    """
    post_count = 0
    for thd in threads:
        link = thd['link']
        response = get_request(link)
        soup = soupify(response)
        tables = get_tables(soup)
        insert_users(tables)
        insert_posts(tables)
        post_count += len(tables)
        # check if a thread has more than one page
        nav_page = soup.find('div', {'class': 'pagenav'})
        # if a thread has more than one page, connect to the next page and update user and post information 
        # until no more pages are left 
        if nav_page != None:
            next_page = soup.find("a", {"rel": "next"})
            while next_page != None:
                href = next_page['href']
                response = get_request(href)
                soup = soupify(response)
                tables = get_tables(soup)
                insert_users(tables)
                insert_posts(tables)
                #print(href)
                post_count += len(tables)
                next_page = soup.find("a", {"rel": "next"})
        # once the entire thread has been parsed and data uploaded, sleep for a random time (2-5 s) and 
        # repeat for the next thread 
        time.sleep(np.random.randint(2,6))
    print(post_count)

In [76]:
# insert users into user sql table
def insert_users(tables):
    """
    Takes in a list of html elements 
    Updates the sql user table (pc_user)
    Returns None
    ____________________
    pc_user
        users: list of usernames
        member_since: list dates became a member for each user
        num_posts: list of total number of posts for each user
    
    """
    
    for table in tables:
        if  "moderator" in table.find('div', {'class': 'smallfont'}).text.lower():
            continue
        n = 0
        if table.find("a", {"class": "bigusername"}) == None:
            user = table.find("div", {"id": re.compile("postmenu_\d+")}).text.strip()
        else:
            user = table.find("a", {"class": "bigusername"}).text
        
        query_user = ("""
        SELECT users
        FROM   pc_user
        WHERE  users = %s;
        """,(user,)) 

                                cur.execute(query_user[0], query_user[1])
                                tbl_user = cur.fetchone()
                                if tbl_user != None:
                                    if tbl_user[0] == user:
                                        continue
                                if table.find_all("div", {"class":"smallfont"})[1].find("img") != None:
                                    n = 1   
                                if table.find_all("div", {"class":"smallfont"})[0].text.lower() == "guest":
                                    member_date = "nan"
                                else:
                                    member_date = table.findAll('br')[n].findNext('div').text.split(":")[-1].strip()
                                if 'location' in table.findAll('br')[n].findAllNext('div')[1].text.lower():
                                    num_posts = table.findAll('br')[n].findAllNext('div')[2].text
                                    num_posts = re.sub("[^0-9]", "", num_posts)
                                else:
                                    num_posts = table.findAll('br')[n].findAllNext('div')[1].text
                                    num_posts = re.sub("[^0-9]", "", num_posts)
                                if len(num_posts) == 0:
                                    num_posts = "nan"

                                query_insert = ("""
                                INSERT INTO pc_user (users, num_posts, member_since)
                                VALUES (%s, %s, %s);
                                """,
                                (user, num_posts, member_date))


                                cur.execute(query_insert[0], query_insert[1])
                                conn.commit()

        
        
            


In [68]:
def insert_posts(tables):
    """
    Takes in a list of html elements
    Updates sql post table (pc_post)
    Returns None
    ____________________
    pc_post
        users: username for each user
        posts: post for each user
        post_type: author or responder
        url:  url of the page
    
    """

    for i,table in enumerate(tables):
        if table.find("a", {"class": "bigusername"}) == None:
            user = table.find("div", {"id": re.compile("postmenu_\d+")}).text.strip()
        else:
            user = table.find("a", {"class": "bigusername"}).text
        post = table.find_all("div", {"id": re.compile("post_message_\d+")})[0].text.strip()
        post = "".join([letter for letter in post if letter in printable])
        if "http" in post:
            post = re.sub(r'http\S+', '', post)
            post = "".join([letter for letter in post if letter in printable])
        if i == 0:
            post_type = 'author'
        else:
            post_type = 'responder'
        
        query_insert = ("""
        INSERT INTO pc_post (users, posts, post_type)
        VALUES (%s, %s, %s);
        """,
        (user,  post, post_type))
         
        
        cur.execute(query_insert[0], query_insert[1])
        conn.commit()
        
            
        
            
            
        
       
    

  

In [29]:
response = requests.get("https://forums.psychcentral.com/depression/498539-looking-accountability-partners-motivate-each-other-get-things-done-while-depressed-15.html")

In [30]:
soup = soupify(response)

In [31]:
tables = get_tables(soup)

In [47]:
tables[8].find_all("div", {"id": re.compile("post_message_\d+")})[0].text.strip()

''

In [20]:
thread = [{'title': None, 'link': "https://forums.psychcentral.com/depression/498539-looking-accountability-partners-motivate-each-other-get-things-done-while-depressed-15.html"}]

In [77]:
add_to_db(threads)

2580


In [24]:
test = "Congratulations on completing the paperwork. I have the same problem �� currently, i m in the middle of cleaning it up and I have been working on it for two days, and will probably need to dedicate three more days on it. I know how relieved you must feel.Yes, I also want to be self disciplined and deal with paperwork as soon as it arrives. I am sorting out mail from 2007! That is how pathetic i have been and i would like it to stop."

In [31]:
"".join([letter for letter in test if letter in string.printable])

'Congratulations on completing the paperwork. I have the same problem  currently, i m in the middle of cleaning it up and I have been working on it for two days, and will probably need to dedicate three more days on it. I know how relieved you must feel.Yes, I also want to be self disciplined and deal with paperwork as soon as it arrives. I am sorting out mail from 2007! That is how pathetic i have been and i would like it to stop.'

In [34]:
printable = set(string.printable)

In [59]:
punctuation = set(string.punctuation)

In [27]:
import string
string.ascii_lowercase

'abcdefghijklmnopqrstuvwxyz'